# Text-to-SQL demo using Vertex AI's Codey APIs. 
The Vertex AI PaLM API contains the publisher endpoints for Google's Pathways Language Model 2 (PaLM 2), which are large language models (LLMs) that generate text and code in response to natural language prompts. The Codey APIs include three models that generate code, suggest code for code completion, and let developers chat to get help with code-related questions.

In [ ]:
# !pip install SQLAlchemy==2.0.16
# !pip install pymysql==1.0.3
# !pip install cloud-sql-python-connector["pymysql"]==1.2.4
# !pip install google-cloud-aiplatform==1.30.0

In [1]:
from google.cloud import aiplatform
import vertexai
from vertexai.preview.language_models import CodeGenerationModel, CodeChatModel
import google.cloud.sql.connector
from google.cloud.sql.connector import Connector
import sqlalchemy

In [9]:
from importlib.metadata import version
print("google-cloud-aiplatform (vertexai) version:", version('google-cloud-aiplatform'))
print("google-cloud-sql-connector version:", version('cloud-sql-python-connector'))
print("sqlalchemy version:", version('sqlalchemy'))
print("pymysql version:", version('pymysql'))

google-cloud-aiplatform (vertexai) version: 1.30.1
google-cloud-sql-connector version: 1.2.4
sqlalchemy version: 2.0.7
pymysql version: 1.0.3


## Section 1: Loading example dataset into database
1. Download MySQL dump file from [MySQL Public Tutorial](https://www.mysqltutorial.org/mysql-sample-database.aspx)
2. Create Cloud Storage bucket and Cloud SQL MySQL instance
3. Import MySQL dump file into Cloud SQL instance

### Database Schema
This is a very common example relational structure for a retail company
![alt text](images/MySQL-Sample-Database-Schema.png "Title")

### Set environment variables

In [10]:
from config import config
PROJECT_ID="jared-playground"
REGION="us-central1"
INSTANCE_NAME="gen-ai-text-to-mysql"


SQL_DATABASE_URL='https://www.mysqltutorial.org/wp-content/uploads/2018/03/mysqlsampledatabase.zip'
BUCKET = 'gs://gen-ai-storage-bucket'
SQL_FILE = 'mysqlsampledatabase.sql'

SERVICE_ACCOUNT = "p555626409143-m42vov@gcp-sa-cloud-sql.iam.gserviceaccount.com" 

# initialize parameters
INSTANCE_CONNECTION_NAME = f"{PROJECT_ID}:{REGION}:{INSTANCE_NAME}" # i.e demo-project:us-central1:demo-instance
print(f"Your instance connection name is: {INSTANCE_CONNECTION_NAME}")


Your instance connection name is: jared-playground:us-central1:gen-ai-text-to-mysql


In [11]:
DATABASE_VERSION="MYSQL_8_0"
CPU=2
MEMORY="8GB"
STORAGE_SIZE="10GB"
DB_USER = "root"
DB_NAME = "classicmodels"

# # Make the Cloud Storage bucket
# !gsutil mb -l $REGION $BUCKET

# Make the Cloud SQL instance
# !gcloud sql instances create $INSTANCE_NAME \
#   --database-version=$DATABASE_VERSION \
#   --region=$REGION \
#   --cpu=$CPU \
#   --memory=$MEMORY \
#   --storage-size=$STORAGE_SIZE

# # Generic function to download the SQL dump file
# def get_sql_data(sql_database_url):
#     from io import BytesIO
#     from urllib.request import urlopen
#     from zipfile import ZipFile
    
#     with urlopen(SQL_DATABASE_URL) as zipresp:
#         with ZipFile(BytesIO(zipresp.read())) as zfile:
#             zfile.extractall()

# get_sql_data(SQL_DATABASE_URL)

# # Copy the file to the Cloud Storage bucket
# !gsutil cp $SQL_FILE $BUCKET

### Important: Allow the Cloud SQL service account to read from the bucket

In [16]:
# !gsutil iam ch serviceAccount:$SERVICE_ACCOUNT:objectAdmin \
# $BUCKET

Import the SQL dump file into the Cloud SQL instance.

In [17]:
# !gcloud sql import sql $INSTANCE_NAME $BUCKET/$SQL_FILE --quiet

## Section 2: Use Text-to-SQL Generative AI to get questions/answers from our database.
Vertex AI includes a suite of models that work with code. Together these code models are referred to as the Vertex AI Codey APIs. The Vertex AI Codey APIs include the following:

* The **code generation** API - Generates code based on a natural language description of the desired code. For example, it can generate a unit test for a function. The code generation API supports the ```code-bison``` model. For more information about the ```code-bison``` model, see [Code generation model parameters](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/models#code-generation-prompt-parameters).

* The **code chat** API - Can power a chatbot that assists with code-related questions. For example, you can use it for help debugging code. The code chat API supports the ```codechat-bison``` model. For more information about the ```codechat-bison``` model, see [Code chat model parameters](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/models#code-chat-prompt-parameters).

* The **code completion** API - Provides code autocompletion suggestions as you write code. The API uses the context of the code you're writing to make its suggestions. The code completion API supports the ```code-gecko``` model. Use the ```code-gecko``` model to help improve the speed and accuracy of writing code. For more information about the ```code-gecko``` model, see [Code completion model parameters](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/models#code-completion-prompt-parameters).


![alt text](images/codey-table.png "Title")

### Provide a schema prompt that describes the schema of the database. Google's [Text-to-SQL paper](https://arxiv.org/pdf/2306.00739.pdf) describes two methodologies for providing the schema in the prompt:
1. ### **Concise prompt design**: Uses the syntax to describe the database schema, which has advantages of clearly presenting table structure. Example below:
![alt text](images/concise.png "Title")
2. ### **Verbose Pompt Design**: Prompts are based on using natural language to describe database schema, which is closer to the way LLMs were trained. Example below:
![alt text](images/verbose.png "Title")

## Verbose prompt design
* ```sql_gen_task_prompt``` summarizes the task and provides instructions.
* ```schema_prompt``` describes the schema of the database with the tables and column types.
* ```sql_gen_examples``` provides examples of question/answer SQL pairs.

In [18]:
sql_gen_task_prompt = """This is a task converting text into SQL statement. Only use the tables and columns provided. We will first give the dataset schema and then ask a question in text. You are asked to generate SQL statement. """

schema_prompt = """There are 8 tables in database classicmodels. The titles of the tables are: customers, employees, offices, orderdetails, orders, payments, productlines, products. 
Table 1 is customers, and it contains customer’s data. Its column names and types are customerNumber (Type is int), customerName (Type is varchar), contactLastName (Type is varchar), contactFirstName (Type is varchar), phone (Type is varchar), addressLine1 (Type is varchar), addressLine2 (Type is varchar), city (Type is varchar), state (Type is varchar), postalCode (Type is varchar), country (Type is varchar), salesRepEmployeeNumber (Type is int), creditLimit (Type is decimal).
Table 2 is employees, and it contains all employee information as well as the organization structure such as who reports to whom. Its column names and types are employeeNumber (Type is int), lastName (Type is varchar), firstName (Type is varchar), extension (Type is varchar), email (Type is varchar), officeCode (Type is varchar), reportsTo (Type is int), jobTitle (Type is varchar).
Table 3 is offices, and it contains sales office data. Its column names and types are officeCode (Type is varchar), city (Type is varchar), phone (Type is varchar), addressLine1 (Type is varchar), addressLine2 (Type is varchar), state (Type is varchar), country (Type is varchar), postalCode (Type is varchar), territory (Type is varchar).
Table 4 is orderdetails, and it contains sales order line items for each sales order. Its column names and types are orderNumber (Type is int), productCode (Type is varchar), quantityOrdered (Type is int), priceEach (Type is decimal), orderLineNumber (Type is smallint).
Table 5 is orders, and it contains sales orders placed by customers. Its column names and types are orderNumber (Type is int), orderDate (Type is date), requiredDate (Type is date), shippedDate (Type is date), status (Type is varchar), comments (Type is text), customerNumber (Type is int).
Table 6 is payments, and it contains payments made by customers based on their accounts. Its column names and types are customerNumber (Type is int), checkNumber (Type is varchar), paymentDate (Type is date), amount (Type is decimal).
Table 7 is productlines, and it contains a list of product line categories. Its column names and types are productLine (Type is varchar), textDescription (Type os varchar), htmlDescription (Type is mediumtext), image (Type is mediumblob).
Table 8 is products, and it contains a list of scale model cars. Its column names and types are productCode (Type is varchar), productName (Type is varchar), productLine (Type is varchar), productScale (Type is varchar), productVendor (Type is varchar), productDescription (Type is text), quantityInStock (Type is smallint), buyPrice (Type is decimal), MSRP (Type is decimal).
The primary keys are: customerNumber from Table customers, employeeNumber from Table employees,
officeCode from Table offices, orderNumber and productCode from Table orderdetails, orderNumber from Table orders, customerNumber and checkNumber from Table payments, productLine from Table productlines, productCode from Table products.
The foreign keys are: salesRepEmployeeNumber from Table customers is equivalent with employeeNumber from Table employees, officeCode from Table employees is equivalent with officeCode from table offices, reportsTo from Table employees is equivalent with employeeNumber from Table employees, customerNumber from Table orders is equivalent with customerNumber from Table customers, productLine from Table products is equivelent with productLine from Table productLines. Use foreign keys to join Tables. """

sql_gen_examples = """Let us take 4 example questions and turn them into a SQL statements about database classicmodels.  
Here is an example: 
The question is: How many unique customers placed an order between 1/1/2004 and 1/31/2004? 
The corresponding SQL is: SELECT count(distinct t1.customerNumber) as distinct_customers FROM orders t5 INNER JOIN customers t1 ON t5.customerNumber = t1.customerNumber WHERE t5.orderDate BETWEEN '2004-01-01' AND '2004-01-31';
Here is an example: 
The question is: What is the product name and price of the highest priced item that was ever purchased? 
The corresponding SQL is: SELECT t8.productName, t4.priceEach FROM orders t5 INNER JOIN orderdetails t4 ON t5.orderNumber = t4.orderNumber LEFT JOIN products t8 ON t4.productCode = t8.productCode ORDER BY priceEach DESC LIMIT 1;
Here is an example: 
The question is: What is the name of the customer with the highest order amount on 12/1/2003? 
The corresponding SQL is: SELECT orders.customerName, orders.total_orderAmount FROM (SELECT t5.orderNumber, t1.customerNumber, t1.customerName, SUM(t4.quantityOrdered*t4.priceEach) as total_orderAmount FROM customers t1 INNER JOIN orders t5 ON t1.customerNumber = t5.customerNumber INNER JOIN orderdetails t4 ON t5.orderNumber = t4.orderNumber WHERE t5.orderDate = '2003-12-01' GROUP BY 1,2,3) orders ORDER BY total_orderAmount DESC LIMIT 1;
Here is an example: 
The question is: How many unique orders were placed at the Paris office during the year 2004?
The corresponding SQL is: SELECT count(distinct t5.orderNumber) as orders FROM orders t5 INNER JOIN customers t1 ON t5.customerNumber = t1.customerNumber LEFT JOIN employees t2 ON t1.salesRepEmployeeNumber = t2.employeeNumber LEFT JOIN offices t3 ON t2.officeCode = t3.officeCode WHERE YEAR(t5.orderDate) = 2004 AND t3.city = 'Paris';"""

### Create functions for Generative AI code generation API and MySQL queries
1. **get_sql_from_code_gen**: Takes the following input parameters:
    * ```prompt```: The prompt with context about the MySQL database schema and column types.
    * ```temperature```: The temperature is used for sampling during response generation. Temperature controls the degree of randomness in token selection. Lower temperatures are good for prompts that require a more deterministic and less open-ended or creative response, while higher temperatures can lead to more diverse or creative results. A temperature of 0 is deterministic, meaning that the highest probability response is always selected (Acceptable values: 0.0–1.0; Default: 0.2).
    * ```max_output_tokens```: Maximum number of tokens that can be generated in the response. A token is approximately four characters. 100 tokens correspond to roughly 60-80 words. Specify a lower value for shorter responses and a higher value for longer responses (Acceptable values: 1–2048; Default: 1024).
    * ```model```: ```code-bison@001``` is the name of the model that supports code generation. It's a foundation model that generates code based on a natural language description. The type of content that ```code-bison``` can create includes functions, web pages, and unit tests. ```code-bison``` is supported by the code generation Codey APIs. Codey APIs are in the PaLM API family. Release date: June 29, 2023.
2. **test_sql**: Takes a SQL query as input and attempts to run a query against a Cloud SQL MySQL database.
    * ```query```: The query to run. This can be fed directly from the previous code generation function.

In [19]:
def get_sql_from_code_gen(project: str, region: str, prompt: str, temperature: float = 0.2, max_output_tokens: int = 1024, model: str = 'code-bison@001'):
    
    vertexai.init(project=project, location=region)
    parameters = {
        "temperature": temperature,
        "max_output_tokens": max_output_tokens
    }
    code_gen_model = CodeGenerationModel.from_pretrained(model)
    
    response = code_gen_model.predict(
        prefix = prompt,
        **parameters
    )

    return (response.text)
    



def test_sql(query: str):
    
    # initialize Connector object
    connector = Connector()

    # function to return the database connection object
    def getconn():
        conn = connector.connect(
            INSTANCE_CONNECTION_NAME,
            "pymysql",
            user=DB_USER,
            db=DB_NAME
        )
        return conn

    # create connection pool with 'creator' argument to our connection object function
    pool = sqlalchemy.create_engine(
        "mysql+pymysql://",
        creator=getconn,
    )

    # connect to connection pool
    with pool.connect() as db_conn:
        # query and fetch ratings table
        results = db_conn.execute(sqlalchemy.text(f"{query};")).fetchall()
        # show results
        output_list = []
        for row in results:
            output_list.append(row)
         
    connector.close()
    return output_list

### Create a combined prompt and request a SQL query
* ```text_to_sql``` This is the question which we want to get a SQL query.
* ```verbose_combined_prompt``` Combines all prompt subcomponents into one to make the API call.

In [20]:
text_to_sql = """What were sales for the year 2005 by state filtered by customers based in USA?"""
verbose_combined_prompt = f"""{sql_gen_task_prompt}

{schema_prompt}

{sql_gen_examples}

The test question to be answered is: {text_to_sql}

The corresponding SQL is: """
print(f"""Prompt to send to code gen API: \n{verbose_combined_prompt}""")

verbose_sql_query = get_sql_from_code_gen(project = PROJECT_ID, 
                      region = REGION,
                      prompt = verbose_combined_prompt,
                      temperature = 0.2, 
                      max_output_tokens = 1024, 
                      model = "code-bison@001")
print(verbose_sql_query)

Prompt to send to code gen API: 
This is a task converting text into SQL statement. Only use the tables and columns provided. We will first give the dataset schema and then ask a question in text. You are asked to generate SQL statement. 

There are 8 tables in database classicmodels. The titles of the tables are: customers, employees, offices, orderdetails, orders, payments, productlines, products. 
Table 1 is customers, and it contains customer’s data. Its column names and types are customerNumber (Type is int), customerName (Type is varchar), contactLastName (Type is varchar), contactFirstName (Type is varchar), phone (Type is varchar), addressLine1 (Type is varchar), addressLine2 (Type is varchar), city (Type is varchar), state (Type is varchar), postalCode (Type is varchar), country (Type is varchar), salesRepEmployeeNumber (Type is int), creditLimit (Type is decimal).
Table 2 is employees, and it contains all employee information as well as the organization structure such as who 

### Test the SQL query

In [21]:
verbose_test_results = test_sql(verbose_sql_query)
print(verbose_test_results)

[(Decimal('378973.82'), 'CA'), (Decimal('5849.44'), 'PA'), (Decimal('28500.78'), 'CT'), (Decimal('117730.73'), 'MA'), (Decimal('43525.04'), 'NJ')]


### Make corrections

In [22]:
corrected_sql_query = '''
SELECT *
FROM orders
JOIN customers ON customers.customerNumber  = orders.customerNumber
WHERE YEAR(orderdate) = "2005"
AND country = "USA"
LIMIT 1
'''
corrected_results = test_sql(corrected_sql_query)
print(corrected_results)

[(10368, datetime.date(2005, 1, 19), datetime.date(2005, 1, 27), datetime.date(2005, 1, 24), 'Shipped', 'Can we renegotiate this one?', 124, 124, 'Mini Gifts Distributors Ltd.', 'Nelson', 'Susan', '4155551450', '5677 Strong St.', None, 'San Rafael', 'CA', '97562', 'USA', 1165, Decimal('210500.00'))]


## Concise prompt design
In order to get the concise schema design, we first need to query the INFORMATION_SCHEMA views to get the table and column details

In [23]:
schema_value_generator = '''
SELECT CONCAT('[Schema (values)]: ', '| ', table_schema, ' | ', GROUP_CONCAT(table_name, ' : ', table_column SEPARATOR ' | '), ';') as table_column
FROM (
SELECT table_schema, table_name, GROUP_CONCAT(column_name SEPARATOR ' , ') as table_column
FROM  INFORMATION_SCHEMA.columns
WHERE table_schema = @schema
GROUP BY table_schema, table_name
ORDER BY table_name) col
GROUP BY table_schema;
'''

column_name_generator = '''
SELECT CONCAT('[Column names (type)]: ', GROUP_CONCAT(CONCAT_WS('',table_name, ' : ',column_name,' (',data_type,')') SEPARATOR ' | '), ';') AS table_column
FROM  INFORMATION_SCHEMA.columns
WHERE table_schema = @schema
ORDER BY table_column;
'''

primary_key_generator = '''
SELECT CONCAT('[Primary Keys]: ', GROUP_CONCAT(CONCAT_WS(' : ', table_name , primary_key) SEPARATOR ' | '), ';') as table_column
FROM (
  SELECT table_name, GROUP_CONCAT(column_name) AS primary_key
  FROM INFORMATION_SCHEMA.KEY_COLUMN_USAGE
  WHERE TABLE_SCHEMA=@schema
  AND CONSTRAINT_NAME='PRIMARY'
  GROUP BY table_name, ordinal_position
  ORDER BY table_name, primary_key) pk1;
'''

foreign_key_generator = '''
SELECT CONCAT('[Foreign Keys]: ', GROUP_CONCAT(CONCAT(TABLE_NAME, ' : ',  COLUMN_NAME, ' equals ', REFERENCED_TABLE_NAME, ' : ', REFERENCED_COLUMN_NAME) SEPARATOR ' | ')) AS table_column
FROM INFORMATION_SCHEMA.KEY_COLUMN_USAGE
WHERE TABLE_SCHEMA=@schema 
AND CONSTRAINT_NAME LIKE '%ibfk%'
ORDER BY table_column;
'''

### Create a function to query the MySQL INFORMATION_SCHEMA views.

In [24]:
def schema_generator(query: str, schema: str = 'classicmodels', group_concat_max_len: int = 102400):
    

    # initialize Connector object
    connector = Connector()

    # function to return the database connection object
    def getconn():
        conn = connector.connect(
            INSTANCE_CONNECTION_NAME,
            "pymysql",
            user=DB_USER,
            db=DB_NAME
        )
        return conn

    # create connection pool with 'creator' argument to our connection object function
    pool = sqlalchemy.create_engine(
        "mysql+pymysql://",
        creator=getconn,
    )

    # connect to connection pool
    with pool.connect() as db_conn:
        db_conn.execute(sqlalchemy.text(f"SET @schema='{schema}';"))
        db_conn.execute(sqlalchemy.text(f"SET SESSION group_concat_max_len = {group_concat_max_len};"))
        # query and fetch ratings table
        results = db_conn.execute(sqlalchemy.text(query)).fetchall()
        # show results
        output_list = []
        for row in results:
            output_list.append(row.table_column)
        
    return output_list[0]

schema_values = schema_generator(schema_value_generator)
column_name_values = schema_generator(column_name_generator)
primary_key_values = schema_generator(primary_key_generator)
foreign_key_values = schema_generator(foreign_key_generator)

### Create a combined prompt and request a SQL query

In [25]:
text_to_sql = """What were sales for the year 2005 by state filtered by customers based in USA?"""
concise_combined_prompt = f"""This is a task converting text into SQL statement. We will first given the dataset schema and then ask a question in text. You are asked to generate SQL statement.
Here is the schema:
{schema_values}

{column_name_values}

{primary_key_values}

{foreign_key_values}

{sql_gen_examples}

The question is: {text_to_sql}
The corresponding SQL is: 
"""
print(f"""Prompt to send to code gen API: \n{concise_combined_prompt}""")

concise_sql_query = get_sql_from_code_gen(project = PROJECT_ID, 
                      region = REGION,
                      prompt = concise_combined_prompt,
                      temperature = 0.2, 
                      max_output_tokens = 1024, 
                      model = "code-bison@001")
print(concise_sql_query)

Prompt to send to code gen API: 
This is a task converting text into SQL statement. We will first given the dataset schema and then ask a question in text. You are asked to generate SQL statement.
Here is the schema:
[Schema (values)]: | classicmodels | customers : customerNumber , customerName , contactLastName , contactFirstName , phone , addressLine1 , addressLine2 , city , state , postalCode , country , salesRepEmployeeNumber , creditLimit | employees : employeeNumber , lastName , firstName , extension , email , officeCode , reportsTo , jobTitle | offices : officeCode , city , phone , addressLine1 , addressLine2 , state , country , postalCode , territory | orderdetails : orderNumber , productCode , quantityOrdered , priceEach , orderLineNumber | orders : orderNumber , orderDate , requiredDate , shippedDate , status , comments , customerNumber | payments : customerNumber , checkNumber , paymentDate , amount | productlines : productLine , textDescription , htmlDescription , image | p

### Test the SQL query

In [26]:
concise_test_results = test_sql(concise_sql_query)
print(concise_test_results)

[('CA', Decimal('378973.82')), ('PA', Decimal('5849.44')), ('CT', Decimal('28500.78')), ('MA', Decimal('117730.73')), ('NJ', Decimal('43525.04'))]


### Make corrections

In [27]:
corrected_sql_query = '''
SELECT *
FROM orders
JOIN customers ON customers.customerNumber  = orders.customerNumber
WHERE YEAR(orderdate) = "2005"
AND country = "USA"
LIMIT 1
'''
corrected_results = test_sql(corrected_sql_query)
print(corrected_results)

[(10368, datetime.date(2005, 1, 19), datetime.date(2005, 1, 27), datetime.date(2005, 1, 24), 'Shipped', 'Can we renegotiate this one?', 124, 124, 'Mini Gifts Distributors Ltd.', 'Nelson', 'Susan', '4155551450', '5677 Strong St.', None, 'San Rafael', 'CA', '97562', 'USA', 1165, Decimal('210500.00'))]


## Section 3 (**BONUS**): Let's interact with the Code Chat API to see if it can generate questions for us.
* This could be useful for generating a lot of questions/answers about the database and creating embeddings of the questions. A vector database like Vertex Matching Engine could store the question embeddings for future similarity matching and query retrieval/execution.
* Additionally, while fine-tuning code-bison is not possible today it is in the roadmap and this workflow could be used to generate lots of example question/answers.

Environment variables

In [28]:
TEMPERATURE = 0.2
MAX_OUTPUT_TOKENS = 1024
CHAT_MODEL = "codechat-bison@001"

Initialize model

In [29]:
vertexai.init(project=PROJECT_ID, location=REGION)
parameters = {
    "temperature": TEMPERATURE,
    "max_output_tokens": MAX_OUTPUT_TOKENS
}
code_chat_model = CodeChatModel.from_pretrained(CHAT_MODEL)

In [30]:
prefix_schema_prompt = """Generate a list of 10 questions about this schema that can be answered with SQL queries:"""

In [31]:
print(f"""{prefix_schema_prompt}
    {schema_prompt}
    The questions are: """)

Generate a list of 10 questions about this schema that can be answered with SQL queries:
    There are 8 tables in database classicmodels. The titles of the tables are: customers, employees, offices, orderdetails, orders, payments, productlines, products. 
Table 1 is customers, and it contains customer’s data. Its column names and types are customerNumber (Type is int), customerName (Type is varchar), contactLastName (Type is varchar), contactFirstName (Type is varchar), phone (Type is varchar), addressLine1 (Type is varchar), addressLine2 (Type is varchar), city (Type is varchar), state (Type is varchar), postalCode (Type is varchar), country (Type is varchar), salesRepEmployeeNumber (Type is int), creditLimit (Type is decimal).
Table 2 is employees, and it contains all employee information as well as the organization structure such as who reports to whom. Its column names and types are employeeNumber (Type is int), lastName (Type is varchar), firstName (Type is varchar), extension (T

In [32]:
chat = code_chat_model.start_chat()

response = chat.send_message(
        f"""{prefix_schema_prompt}
        {schema_prompt}
        The questions are: """, **parameters
)
    
response_text = response.text
print(f"Response from Model: {response_text}")

Response from Model: Here are 10 questions that can be answered with SQL queries on this schema:

1. What is the total sales for each product line?
2. Which employee has the highest salary?
3. What is the average order size?
4. Which customer has placed the most orders?
5. What is the most popular product?
6. Which office has the most employees?
7. What is the most expensive product?
8. What is the least expensive product?
9. What is the most recent order?
10. What is the oldest order?
